In [ ]:
from docplex.mp.model import Model
from qiskit_optimization import QuadraticProgram
from qiskit.quantum_info import SparsePauliOp
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import InequalityToEquality, QuadraticProgramToQubo, IntegerToBinary, LinearEqualityToPenalty
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QAOAAnsatz
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms.optimizers import COBYLA
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.visualization import plot_histogram
from qiskit.visualization import plot_coupling_map
from qiskit_ibm_runtime import Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from scipy.optimize import minimize
import rustworkx as rx
from typing import List, Tuple
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from dotenv import load_dotenv
import os


In [ ]:
# Set a seed for future replicability
algorithm_globals.random_seed = 42

In [ ]:
def build_docplex_model():
    # 1) Create the docplex model
    m = Model(name="my_miqp")

    # 2) Define parameters (like in Julia)
    n = 3
    h = [1, 1, 1]
    k = [1.64, 1.64, 1.64]
    lambdas = [1, 1, 1]   # 'lambda' is a reserved keyword in Python
    sigma = [2, 2, 2]
    M = 10
    N = 10
    ST0 = 0
    ST3 = 1

    # 3) Create variables
    # Integer variables
    ST = [m.integer_var(lb=0, ub=3, name=f"ST_{i}") for i in range(1,3)] # ST[0] and ST[3] are constants
    z  = [m.integer_var(lb=0, ub=3, name=f"z_{i}")  for i in range(1, n+1)]
    u  = [m.integer_var(lb=0, ub=3, name=f"u_{i}")  for i in range(1, n+1)]
    H  = [m.integer_var(lb=0, ub=3, name=f"H_{i}")  for i in range(1, n+1)]
    # Binary variables
    y11 = m.binary_var(name="y_1_1")
    y12 = m.binary_var(name="y_1_2")
    y13 = m.binary_var(name="y_1_3")
    x10 = m.binary_var(name="x_1_0")
    x21 = m.binary_var(name="x_2_1")
    x32 = m.binary_var(name="x_3_2")

    # 4) Add constraints that must hold exactly:
    #    ST[3] == 1, ST[0] == 0
    # m.add_constraint(ST[3] == 1)
    # m.add_constraint(ST[0] == 0)

    # 5) Build the objective expression
    #    In DOcplex, you can just write z[i]*z[i] for z[i]^2, etc.
    #    However, be mindful that squares of integer variables => MIQP
    #    Below is a direct translation from your Julia objective.

    # Minimize sum(h[i]*k[i]*sigma[i]*z[i]) for i in 1..n
    obj_expr = 0
    for i in range(n):
        obj_expr += h[i] * k[i] * sigma[i] * z[i]
    m.minimize(obj_expr)

    # 6) Add the constraints
    # u[1] <= lambda[1] - z[1]^2 + M*(1-y11)
    m.add_constraint(u[0] <= lambdas[0] - z[0]**2 + M*(1 - y11))

    # y11 >= 1  => forces y11==1
    m.add_constraint(y11 >= 1)

    # u[2] <= lambda[1] - z[1]^2 + lambda[2] - z[2]^2 + M*(1-y12)
    m.add_constraint(u[1] <= lambdas[0] - z[0]**2 + lambdas[1] - z[1]**2 + M*(1 - y12))

    # y12 >= 1
    m.add_constraint(y12 >= 1)

    # u[3] <= lambda[1] - z[1]^2 + lambda[2] - z[2]^2 + lambda[3] - z[3]^2 + M*(1-y13)
    m.add_constraint(u[2] <= lambdas[0] - z[0]**2 +
                                lambdas[1] - z[1]**2 +
                                lambdas[2] - z[2]**2 + M*(1 - y13))
    
    # y13 >= 1
    m.add_constraint(y13 >= 1)

    # z[1]^2 + z[2]^2 + z[3]^2 >= sum(lambda) - ST[3]
    m.add_constraint(z[0]**2 + z[1]**2 + z[2]**2 >= sum(lambdas) - ST3)

    # z[1]^2 == H[1] + lambda[1] - ST[1]
    m.add_constraint(z[0]**2 == H[0] + lambdas[0] - ST[0])

    # H[1] >= ST[0]
    m.add_constraint(H[0] >= ST0)

    # H[1] - ST[0] - N*(1 - x10) <= 0
    m.add_constraint(H[0] - ST0 - N*(1 - x10) <= 0)

    # x10 >= 1
    m.add_constraint(x10 >= 1)

    # z[2]^2 == H[2] + lambda[2] - ST[2]
    m.add_constraint(z[1]**2 == H[1] + lambdas[1] - ST[1])

    # H[2] >= ST[1]
    m.add_constraint(H[1] >= ST[0])

    # H[2] - ST[1] - N*(1 - x21) <= 0
    m.add_constraint(H[1] - ST[0] - N*(1 - x21) <= 0)

    # x21 >= 1
    m.add_constraint(x21 >= 1)

    # z[3]^2 == H[3] + lambda[3] - ST[3]
    m.add_constraint(z[2]**2 == H[2] + lambdas[2] - ST3)

    # H[3] >= ST[2]
    m.add_constraint(H[2] >= ST[1])

    # H[3] - ST[2] - N*(1 - x32) <= 0
    m.add_constraint(H[2] - ST[1] - N*(1 - x32) <= 0)

    # x32 >= 1
    m.add_constraint(x32 >= 1)

    return m


In [ ]:
# 1. Build the DOcplex model and convert it to a QuadraticProgram.
docplex_model = build_docplex_model()
qp = from_docplex_mp(docplex_model)
print("=== Original Quadratic Program ===")
print(qp.export_as_lp_string())

In [ ]:
 # ----- Step 2. Use converters to remove constraints -----
# First, convert any inequality constraints into equality constraints.
ineq_to_eq = InequalityToEquality()
qp_eq = ineq_to_eq.convert(qp)

# Then, convert linear equality constraints into penalty terms in the objective.
lin_eq_to_penalty = LinearEqualityToPenalty()
qp_unconstrained = lin_eq_to_penalty.convert(qp_eq)

# (For many MIQPs it is also useful to convert integer variables to binary and then to a QUBO.)
int_2_bin = IntegerToBinary()
qp_bin = int_2_bin.convert(qp_unconstrained)
qp_qubo = QuadraticProgramToQubo().convert(qp_bin)

print("\n=== QUBO Formulation ===")
print(qp_qubo.export_as_lp_string())

In [ ]:
num_binary = qp_qubo.get_num_vars()
print("Number of binary variables:", num_binary)

In [ ]:
# ----- Step 2. Convert QUBO to an Ising operator -----
# The QUBO can be mapped to an Ising Hamiltonian.
qubit_op, offset = qp_qubo.to_ising()


In [ ]:
# ----- Step 3. Build and display the QAOAAnsatz circuit -----
# Create the QAOAAnsatz using the operator and choose 1 repetition.
cirquit_qaoa_ansatz = QAOAAnsatz(cost_operator=qubit_op, reps=1)
print("\n=== QAOAAnsatz Circuit ===")

# Finalize the cirquit
cirquit_qaoa_ansatz.measure_all()

# Display the circuit as a matplotlib figure in the notebook.
cirquit_qaoa_ansatz.draw('mpl', fold=False, scale=0.2, idle_wires=False)

In [ ]:
# # Initialize the backend
# from qiskit.providers.fake_provider import Fake127QPulseV1 # or any other Fake device
# chip = Fake127QPulseV1()
# #backend = AerSimulator.from_backend(backend=chip, max_memory_mb=20480) # 20GB of RAM
# backend = AerSimulator(max_memory_mb=20480, method="matrix_product_state")


In [ ]:
# Load IQP_API_TOKEN from the .env file
load_dotenv()
token = os.getenv("IQP_API_TOKEN")

# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<MY_IBM_QUANTUM_TOKEN>", overwrite=True, set_as_default=True)
service = QiskitRuntimeService(channel='ibm_quantum', token=token)
backend = service.least_busy(min_num_qubits=127)
print(backend)

In [ ]:
# Plot the coupling map (topology) graphically
plot_coupling_map(num_qubits=127, qubit_coordinates=None, coupling_map=backend.configuration().coupling_map)


In [ ]:
# Optimize the cirquit
pm = generate_preset_pass_manager(optimization_level=3,
                                    backend=backend)

candidate_circuit = pm.run(cirquit_qaoa_ansatz)
candidate_circuit.draw('mpl', fold=False, scale=0.1, idle_wires=False)

In [ ]:
initial_gamma = np.pi
initial_beta = np.pi/2
init_params = [initial_gamma, initial_beta]

In [ ]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):

    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)


    return cost

In [ ]:
# Check the cirquit parameters
print(candidate_circuit.num_parameters)
print(candidate_circuit.parameters)


In [ ]:
objective_func_vals = [] # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    # estimator = Estimator(mode=backend)
    estimator = Estimator(mode=backend)

    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, qubit_op, estimator),
        method="COBYLA",
    )
    print(result)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

In [ ]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
optimized_circuit.draw('mpl', fold=False, idle_wires=False)

In [ ]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"


pub= (optimized_circuit, )
job = sampler.run([pub], shots=10000)

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val/shots for key, val in counts_int.items()}
final_distribution_bin = {key: val/shots for key, val in counts_bin.items()}
print(final_distribution_bin)

In [ ]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]

In [ ]:
_PARITY = np.array([-1 if bin(i).count("1") % 2 else 1 for i in range(256)], dtype=np.complex128)


def evaluate_sparse_pauli(state: int, observable: SparsePauliOp) -> complex:
    """Utility for the evaluation of the expectation value of a measured state."""
    packed_uint8 = np.packbits(observable.paulis.z, axis=1, bitorder="little")
    state_bytes = np.frombuffer(state.to_bytes(packed_uint8.shape[1], "little"), dtype=np.uint8)
    reduced = np.bitwise_xor.reduce(packed_uint8 & state_bytes, axis=1)
    return np.sum(observable.coeffs * _PARITY[reduced])

def best_solution(samples, hamiltonian):
    """Find solution with lowest cost"""
    min_cost = 1000
    min_sol = None
    for bit_str in samples.keys():
        # Qiskit use little endian hence the [::-1]
        candidate_sol = int(bit_str)
        # fval = qp.objective.evaluate(candidate_sol)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        if fval <= min_cost:
            min_sol = candidate_sol

    return min_sol

best_sol = best_solution(final_distribution_int, qubit_op)
best_sol_bitstring = to_bitstring(int(best_sol), num_binary)
best_sol_bitstring.reverse()

print("Result bitstring:", best_sol_bitstring)

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt

# auxiliary function to help plot cumulative distribution functions
def _plot_cdf(objective_values: dict, ax, color):
    x_vals = sorted(objective_values.keys(), reverse=True)
    y_vals = np.cumsum([objective_values[x] for x in x_vals])
    ax.plot(x_vals, y_vals, color=color)


def plot_cdf(dist, ax, title):
    _plot_cdf(dist, ax, "C1",)
    ax.vlines(min(list(dist.keys())), 0, 1, "C1", linestyle="--")

    ax.set_title(title)
    ax.set_xlabel("Objective function value")
    ax.set_ylabel("Cumulative distribution function")
    ax.grid(alpha=0.3)

# auxiliary function to convert bit-strings to objective values
def samples_to_objective_values(samples, hamiltonian):
    """Convert the samples to values of the objective function."""

    objective_values = defaultdict(float)
    for bit_str, prob in samples.items():
        candidate_sol = int(bit_str)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        objective_values[fval] += prob

    return objective_values

In [ ]:
result_dist = samples_to_objective_values(final_distribution_int, qubit_op)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_cdf(result_dist, ax, "Eagle  device")

In [ ]:
# 1. Extract the most likely bitstring:
most_likely_bitstring = max(final_distribution_bin, key=final_distribution_bin.get)
print("Most likely bitstring:", most_likely_bitstring)

In [ ]:
# 2. Convert the bitstring into a list of integers.
binary_solution = [int(bit) for bit in most_likely_bitstring]
print("Binary solution (list):", binary_solution)

In [ ]:
# 3. Construct a dictionary mapping variable names to values.
#    The QUBO object keeps track of the variable ordering.
binary_assignment = {
    var: binary_solution[i] for i, var in enumerate(qp_qubo.variables)
}
print("Binary assignment dict:", binary_assignment)

In [ ]:
# 4. Now invert the binary solution to recover original integer variable values.
#    This uses the same IntegerToBinary converter instance used for conversion.
original_values = int_2_bin.interpret(binary_solution)
# Now get the list of original variable names from your original QuadraticProgram
original_variables = qp_eq.variables

# Create a dictionary mapping each variable name to its recovered value
original_solution_dict = {var.name: val for var, val in zip(original_variables, original_values)}

original_solution_int = {var: int(val) for var, val in original_solution_dict.items()}
print("Original variable assignments:")
print(original_solution_int)


In [ ]:
# Assuming final_distribution_bin is a dictionary of bitstrings and their probabilities
final_bits = final_distribution_bin

# Sort bitstrings by probability in descending order
sorted_bits = dict(sorted(final_bits.items(), key=lambda item: item[1], reverse=True))

# Keep only the top 100 most likely values
top_100_bits = dict(list(sorted_bits.items())[:100])

# Get the probabilities
values = np.array(list(top_100_bits.values()))

# Identify the top 4 values
top_4_values = sorted(values, reverse=True)[:4]

# Find the positions of the top 4 values in the sorted data
positions = []
for value in top_4_values:
    positions.append(np.where(values == value)[0][0])  # Taking first match

# Plotting
plt.figure(figsize=(11, 6))
plt.xticks(rotation=45)
plt.title("Result Distribution (Top 100 Bitstrings)")
plt.xlabel("Bitstrings (reversed)")
plt.ylabel("Probability")

# Create a bar chart
bars = plt.bar(list(top_100_bits.keys()), list(top_100_bits.values()), color="tab:grey")

# Highlight the top 4 values in purple
for p in positions:
    bars[p].set_color("tab:purple")

plt.show()
